# Яндекс Музыка

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс Музыки вы сравните поведение пользователей двух столиц.

**Цель исследования** — проверьте три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные о поведении пользователей вы получите из файла `yandex_music_project.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Вы проверите данные на ошибки и оцените их влияние на исследование. Затем, на этапе предобработки вы поищете возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

## Оглавление:
* [Этап 1. Обзор данных](#Обзор_данных)
* [Этап 2. Предобработка данных](#Предобработка_данных)
    *  [Стиль заголовков](#Стиль_заголовков)
    *  [Пропуски значений](#Пропуски_значений)
    *  [Дубликаты](#Дубликаты)
* [Этап 3. Проверка гипотез](#Проверка_гипотез)
    * [Активность пользователей зависит от дня недели](#а_п)
    * [Москва и Петербург предпочитают разные жанры музыки](#м_п)
    * [Разные жанры преобладают в разное время](#р_ж)
* [Итоги исследования](#и_и)

## Этап 1. Обзор данных <a class="anchor" id="Обзор_данных"></a>

Составим первое представление о данных Яндекс Музыки

Импортируем библиотеку `pandas`

In [1]:
import pandas as pd

Считаем данные из файла `datasets/yandex_music_project.csv` и сохраним их в переменную `df`

In [2]:
df = pd.read_csv('datasets/yandex_music_project.csv')

Выведем на экран первые десять строк таблицы:

In [3]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


Одной командой получим общую информацию о таблице:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.е значения.

**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. Предварительно можно утверждать, что данных достаточно для проверки гипотез. Однако в названиях колонок видны нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Используется верблюжий регистр вместо змеиного

Перед тем как приступить к анализу гипотез, необходимо исправить эти ошибки. Так же видны пропуски в колонках `Track`, `artist` и `genre`, поэтому необходимо провести предобработку данных.

## Этап 2. Предобработка данных <a class="anchor" id="Предобработка_данных"></a>

Исправим стиль в заголовках столбцов, исключим пропуски. Затем проверим данные на дубликаты.

### Стиль заголовков <a class="anchor" id="Стиль_заголовков"></a>

Выведем на экран названия столбцов:

In [5]:
col_names = list(df.columns)
col_names

['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day']

Приведём названия в соответствие с хорошим стилем:
* все названия запишем в «змеином_регистре»,
* все символы сделаем строчными,
* устраним пробелы.

Для этого переименуем колонки так:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.' → 'day'.

In [6]:
df = df.set_axis(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], axis='columns')

Проверим результат

In [7]:
list(df.columns)

['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day']

Приступим к пропускам

### Пропуски значений <a class="anchor" id="Пропуски_значений"></a>

Сначала посчитаем, сколько в таблице пропущенных значений.

In [8]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

In [9]:
df.head()

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


Не все пропущенные значения влияют на исследование. Так в `track_name` и `artist_name` пропуски не важны для вашей работыи и достаточно заменить их явными обозначением `unknown`.

Но пропуски в `gen_namere` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями;
* оценить, насколько они повредят расчётам. 

In [10]:
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

Убедимся, что пропусков не осталось.

In [11]:
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

Пруски устранены, теперь займёмся дубликатами.

### Дубликаты <a class="anchor" id="Дубликаты"></a>

Посчитаем количество явных дубликатов.

In [12]:
df.duplicated().sum()

3826

Удалим их.

In [13]:
df = df.drop_duplicates().reset_index(drop = True)

In [14]:
df.duplicated().sum()

0

Теперь обратим внимание на неявные дубликаты. Наличие таковых наиболее вероятно в столцбе `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования. Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке.

In [15]:
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Видны три дубликата жанра `hiphop`: `hip`, `hop`, `hip-hop`. Приведём их к одному виду `hiphop`. Так же присутствует написание жанра на кирилице: `электроника`. Заменим на `electronic`.

In [16]:
def replace_genres(wrong_genres, correct_genre):
    for genre in wrong_genres:
        df['genre'] = df['genre'].replace(genre, correct_genre)

In [17]:
replace_genres(['hip', 'hop', 'hip-hop'], 'hiphop')
replace_genres(['электроника'], 'electronic')

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Вы исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения вы заменили на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## Этап 3. Проверка гипотез <a class="anchor" id="Проверка_гипотез"></a>

### Активность пользователей зависит от дня недели <a class="anchor" id="а_п"></a>

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга.
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Оценим активность пользователей в каждом городе. Сгруппируем данные по городу и посчитаем прослушивания в каждой группе.

In [18]:
df.groupby('city')['user_id'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64

В Москве прослушиваний больше, чем в Петербурге, однако из этого нельщя сделать вывод, что москвичи чаще слушают музыку, потому что в Москве проживает больше людей, чем в Петербургее.

Теперь сгруппируем данные по дню недели и посчитаем прослушивания в понедельник, среду и пятницу.

In [19]:
df.groupby('day')['track'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

Чаще всего слушают музыку в понедельник и пятницу.

Теперь посмотрим на сводные данные для обоих городов и трёх дней недели. Для этого создадим функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города.

In [20]:
def number_tracks(day, city):
    track_list = df[(df['day'] == day) & (df['city'] == city)] 
    track_list_count = track_list['user_id'].count()
    return track_list_count

Занесём данные в датафрейм.

In [21]:
city_list = df['city'].unique()
day_list = df['day'].unique()

city_day_df = pd.DataFrame()

for city in city_list:
    for day in day_list:
        city_day_df.loc[city, day] = number_tracks(day, city)

city_day_df

,Wednesday,Friday,Monday
Saint-Petersburg,7003.0,5895.0,5614.0
Moscow,11056.0,15945.0,15740.0


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Разные жанры преобладают в разное время  <a class="anchor" id="р_ж"></a>

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [22]:
moscow_general = df[df['city'] == 'Moscow']
spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию `genre_weekday()`, которая будет возвращать информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [23]:
def genre_weekday(df, day, time1, time2):
    genre_df = df[(df['day'] == day) & (df['time'] > time1) & (df['time'] < time2)]
    genre_df = genre_df.groupby('genre')['genre'].count().sort_values(ascending=False)
    return genre_df[:10]

In [24]:
genre_weekday(moscow_general, 'Monday', '07:00:00', '11:00:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [25]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [26]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [27]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Москва и Петербург предпочитают разные жанры музыки <a class="anchor" id="м_п"></a>

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве. А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`. Затем отсортируем результат в порядке убывания и сохраним его в таблице `moscow_genres`.

In [28]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Повторим то же и для Петербурга.

In [29]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы** 

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.


## Итоги исследования <a class="anchor" id="и_и"></a>

Мы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.